# Project - Walker2DBulletEnv with Soft Actor Critic (SAC)

In [1]:
import gym
import pybullet_envs
import pybullet_envs.bullet as bul
import numpy as np
import torch
from sac_agent import soft_actor_critic_agent
from replay_memory import ReplayMemory

seed=0  
env = gym.make('Walker2DBulletEnv-v0')

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## import os
## os.environ['PYTHONHASHSEED']=str(seed)

np.random.seed(seed)
env.seed(seed)
max_steps = env._max_episode_steps
print('max_steps: ', max_steps)

batch_size=256

LEARNING_RATE=0.0003 
eval=True  ## 
start_steps=10000 ## Steps sampling random actions
replay_size=1000000 ## size of replay buffer
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# alpha=0.2  # relative importance of the entropy
# gamma=0.99  # discount factor 
# tau=0.005  # target smoothing coefficient(τ)

agent = soft_actor_critic_agent(env.observation_space.shape[0], env.action_space, \
        device=device, hidden_size=256, seed=seed, \
        lr=LEARNING_RATE, gamma=0.99, tau=0.005, alpha=0.2)

memory = ReplayMemory(seed, replay_size)

print('device: ', device)
print('state dim: ', env.observation_space.shape[0])
print('action dim: ', env.action_space)
print('leraning rate: ', LEARNING_RATE)\

threshold = env.spec.reward_threshold

print('threshold: ', threshold)

current_dir=/home/jaeyoung/anaconda3/envs/ee619/lib/python3.8/site-packages/pybullet_envs/bullet


/home/jaeyoung/anaconda3/envs/ee619/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


max_steps:  1000
device:  cuda:1
state dim:  22
action dim:  Box(-1.0, 1.0, (6,), float32)
leraning rate:  0.0003
threshold:  2500.0


In [2]:
def save(agent, directory, filename, episode, reward):
    torch.save(agent.policy.state_dict(), '%s/%s_actor_%s_%s.pth' % (directory, filename, episode, reward))
    torch.save(agent.critic.state_dict(), '%s/%s_critic_%s_%s.pth' % (directory, filename, episode, reward))

In [ ]:
import time
from  collections  import deque

def sac_train(max_steps):

    total_numsteps = 0
    updates = 0
    num_episodes = 20000
    updates=0

    time_start = time.time()
    scores_deque = deque(maxlen=100)
    scores_array = []
    avg_scores_array = [] 
    
    for i_episode in range(num_episodes): 
        episode_reward = 0
        episode_steps = 0
        done = False
        state = env.reset()

        for step in range(max_steps):    
            if start_steps > total_numsteps:
                action = env.action_space.sample()  # Sample random action
            else:
                action = agent.select_action(state)  # Sample action from policy

            if len(memory) > batch_size:                
                # Update parameters of all the networks
                agent.update_parameters(memory, batch_size, updates)

                updates += 1

            next_state, reward, done, _ = env.step(action) # Step
            episode_steps += 1
            total_numsteps += 1
            episode_reward += reward

            mask = 1 if episode_steps == env._max_episode_steps else float(not done)

            memory.push(state, action, reward, next_state, mask) # Append transition to memory

            state = next_state
            
            if done:
                break

        scores_deque.append(episode_reward)
        scores_array.append(episode_reward)        
        avg_score = np.mean(scores_deque)
        avg_scores_array.append(avg_score)
        max_score = np.max(scores_deque)
        
        if i_episode % 100 == 0 and i_episode > 0:
            reward_round = round(episode_reward, 2)
            save(agent, 'dir_chk_lr0.0003', 'weights', str(i_episode), str(reward_round))

        s =  (int)(time.time() - time_start)
            
        print("Ep.: {}, Total Steps: {}, Ep.Steps: {}, Score: {:.3f}, Avg.Score: {:.3f}, Max.Score: {:.3f}, Time: {:02}:{:02}:{:02}".\
            format(i_episode, total_numsteps, episode_steps, episode_reward, avg_score, max_score, \
                  s//3600, s%3600//60, s%60))

                    
        if (avg_score > threshold):
            print('Solved environment with Avg Score:  ', avg_score)
            break;
            
    return scores_array, avg_scores_array 

scores, avg_scores = sac_train(max_steps=max_steps)

Ep.: 0, Total Steps: 15, Ep.Steps: 15, Score: 17.314, Avg.Score: 17.314, Max.Score: 17.314, Time: 00:00:00
Ep.: 1, Total Steps: 30, Ep.Steps: 15, Score: 17.531, Avg.Score: 17.423, Max.Score: 17.531, Time: 00:00:00
Ep.: 2, Total Steps: 37, Ep.Steps: 7, Score: 13.101, Avg.Score: 15.982, Max.Score: 17.531, Time: 00:00:00
Ep.: 3, Total Steps: 50, Ep.Steps: 13, Score: 17.964, Avg.Score: 16.478, Max.Score: 17.964, Time: 00:00:00
Ep.: 4, Total Steps: 65, Ep.Steps: 15, Score: 16.382, Avg.Score: 16.459, Max.Score: 17.964, Time: 00:00:00
Ep.: 5, Total Steps: 86, Ep.Steps: 21, Score: 23.526, Avg.Score: 17.637, Max.Score: 23.526, Time: 00:00:00
Ep.: 6, Total Steps: 94, Ep.Steps: 8, Score: 12.529, Avg.Score: 16.907, Max.Score: 23.526, Time: 00:00:00
Ep.: 7, Total Steps: 116, Ep.Steps: 22, Score: 21.940, Avg.Score: 17.536, Max.Score: 23.526, Time: 00:00:00
Ep.: 8, Total Steps: 132, Ep.Steps: 16, Score: 17.669, Avg.Score: 17.551, Max.Score: 23.526, Time: 00:00:00
Ep.: 9, Total Steps: 154, Ep.Steps: 2

Ep.: 76, Total Steps: 1109, Ep.Steps: 12, Score: 14.758, Avg.Score: 16.698, Max.Score: 25.095, Time: 00:00:25
Ep.: 77, Total Steps: 1123, Ep.Steps: 14, Score: 17.330, Avg.Score: 16.706, Max.Score: 25.095, Time: 00:00:25
Ep.: 78, Total Steps: 1136, Ep.Steps: 13, Score: 17.814, Avg.Score: 16.720, Max.Score: 25.095, Time: 00:00:25
Ep.: 79, Total Steps: 1170, Ep.Steps: 34, Score: 25.836, Avg.Score: 16.834, Max.Score: 25.836, Time: 00:00:26
Ep.: 80, Total Steps: 1179, Ep.Steps: 9, Score: 13.522, Avg.Score: 16.793, Max.Score: 25.836, Time: 00:00:26
Ep.: 81, Total Steps: 1189, Ep.Steps: 10, Score: 15.022, Avg.Score: 16.771, Max.Score: 25.836, Time: 00:00:27
Ep.: 82, Total Steps: 1203, Ep.Steps: 14, Score: 18.295, Avg.Score: 16.790, Max.Score: 25.836, Time: 00:00:27
Ep.: 83, Total Steps: 1215, Ep.Steps: 12, Score: 15.938, Avg.Score: 16.780, Max.Score: 25.836, Time: 00:00:27
Ep.: 84, Total Steps: 1228, Ep.Steps: 13, Score: 16.709, Avg.Score: 16.779, Max.Score: 25.836, Time: 00:00:28
Ep.: 85, To

Ep.: 151, Total Steps: 2111, Ep.Steps: 17, Score: 18.760, Avg.Score: 16.409, Max.Score: 25.836, Time: 00:00:53
Ep.: 152, Total Steps: 2120, Ep.Steps: 9, Score: 12.818, Avg.Score: 16.354, Max.Score: 25.836, Time: 00:00:54
Ep.: 153, Total Steps: 2136, Ep.Steps: 16, Score: 18.805, Avg.Score: 16.312, Max.Score: 25.836, Time: 00:00:54
Ep.: 154, Total Steps: 2151, Ep.Steps: 15, Score: 17.924, Avg.Score: 16.354, Max.Score: 25.836, Time: 00:00:55
Ep.: 155, Total Steps: 2157, Ep.Steps: 6, Score: 10.554, Avg.Score: 16.261, Max.Score: 25.836, Time: 00:00:55
Ep.: 156, Total Steps: 2173, Ep.Steps: 16, Score: 18.589, Avg.Score: 16.284, Max.Score: 25.836, Time: 00:00:55
Ep.: 157, Total Steps: 2191, Ep.Steps: 18, Score: 15.971, Avg.Score: 16.235, Max.Score: 25.836, Time: 00:00:56
Ep.: 158, Total Steps: 2212, Ep.Steps: 21, Score: 20.540, Avg.Score: 16.256, Max.Score: 25.836, Time: 00:00:56
Ep.: 159, Total Steps: 2229, Ep.Steps: 17, Score: 18.116, Avg.Score: 16.186, Max.Score: 25.836, Time: 00:00:57
Ep.

Ep.: 225, Total Steps: 3155, Ep.Steps: 8, Score: 12.456, Avg.Score: 16.255, Max.Score: 23.383, Time: 00:01:23
Ep.: 226, Total Steps: 3171, Ep.Steps: 16, Score: 16.503, Avg.Score: 16.237, Max.Score: 23.383, Time: 00:01:23
Ep.: 227, Total Steps: 3186, Ep.Steps: 15, Score: 19.944, Avg.Score: 16.223, Max.Score: 23.383, Time: 00:01:24
Ep.: 228, Total Steps: 3195, Ep.Steps: 9, Score: 12.760, Avg.Score: 16.218, Max.Score: 23.383, Time: 00:01:24
Ep.: 229, Total Steps: 3210, Ep.Steps: 15, Score: 17.122, Avg.Score: 16.266, Max.Score: 23.383, Time: 00:01:24
Ep.: 230, Total Steps: 3226, Ep.Steps: 16, Score: 19.101, Avg.Score: 16.301, Max.Score: 23.383, Time: 00:01:25
Ep.: 231, Total Steps: 3251, Ep.Steps: 25, Score: 22.379, Avg.Score: 16.388, Max.Score: 23.383, Time: 00:01:25
Ep.: 232, Total Steps: 3259, Ep.Steps: 8, Score: 13.318, Avg.Score: 16.361, Max.Score: 23.383, Time: 00:01:26
Ep.: 233, Total Steps: 3275, Ep.Steps: 16, Score: 18.634, Avg.Score: 16.372, Max.Score: 23.383, Time: 00:01:26
Ep.:

Ep.: 299, Total Steps: 4189, Ep.Steps: 8, Score: 13.335, Avg.Score: 16.744, Max.Score: 30.524, Time: 00:01:53
Ep.: 300, Total Steps: 4200, Ep.Steps: 11, Score: 15.123, Avg.Score: 16.747, Max.Score: 30.524, Time: 00:01:53
Ep.: 301, Total Steps: 4212, Ep.Steps: 12, Score: 16.671, Avg.Score: 16.755, Max.Score: 30.524, Time: 00:01:54
Ep.: 302, Total Steps: 4230, Ep.Steps: 18, Score: 19.457, Avg.Score: 16.773, Max.Score: 30.524, Time: 00:01:54
Ep.: 303, Total Steps: 4240, Ep.Steps: 10, Score: 13.524, Avg.Score: 16.728, Max.Score: 30.524, Time: 00:01:55
Ep.: 304, Total Steps: 4253, Ep.Steps: 13, Score: 17.024, Avg.Score: 16.720, Max.Score: 30.524, Time: 00:01:55
Ep.: 305, Total Steps: 4267, Ep.Steps: 14, Score: 16.070, Avg.Score: 16.741, Max.Score: 30.524, Time: 00:01:55
Ep.: 306, Total Steps: 4279, Ep.Steps: 12, Score: 15.366, Avg.Score: 16.774, Max.Score: 30.524, Time: 00:01:56
Ep.: 307, Total Steps: 4288, Ep.Steps: 9, Score: 13.220, Avg.Score: 16.751, Max.Score: 30.524, Time: 00:01:56
Ep.

...

Ep.: 2546, Total Steps: 1150112, Ep.Steps: 1000, Score: 1837.425, Avg.Score: 1604.136, Max.Score: 1897.517, Time: 09:33:52
Ep.: 2547, Total Steps: 1151112, Ep.Steps: 1000, Score: 1863.280, Avg.Score: 1604.904, Max.Score: 1897.517, Time: 09:34:24
Ep.: 2548, Total Steps: 1152112, Ep.Steps: 1000, Score: 1818.186, Avg.Score: 1605.621, Max.Score: 1897.517, Time: 09:34:54
Ep.: 2549, Total Steps: 1153112, Ep.Steps: 1000, Score: 1853.749, Avg.Score: 1606.473, Max.Score: 1897.517, Time: 09:35:25
Ep.: 2550, Total Steps: 1153982, Ep.Steps: 870, Score: 1582.630, Avg.Score: 1604.328, Max.Score: 1897.517, Time: 09:35:49
Ep.: 2551, Total Steps: 1154982, Ep.Steps: 1000, Score: 1826.126, Avg.Score: 1604.736, Max.Score: 1897.517, Time: 09:36:16
Ep.: 2552, Total Steps: 1155612, Ep.Steps: 630, Score: 1117.671, Avg.Score: 1598.409, Max.Score: 1897.517, Time: 09:36:35
Ep.: 2553, Total Steps: 1155993, Ep.Steps: 381, Score: 686.798, Avg.Score: 1587.717, Max.Score: 1897.517, Time: 09:36:47
Ep.: 2554, Total Ste

Ep.: 2614, Total Steps: 1190126, Ep.Steps: 173, Score: 279.205, Avg.Score: 1286.743, Max.Score: 1898.410, Time: 09:53:45
Ep.: 2615, Total Steps: 1190426, Ep.Steps: 300, Score: 473.974, Avg.Score: 1272.907, Max.Score: 1898.410, Time: 09:53:54
Ep.: 2616, Total Steps: 1190632, Ep.Steps: 206, Score: 335.706, Avg.Score: 1258.158, Max.Score: 1898.410, Time: 09:54:01
Ep.: 2617, Total Steps: 1191115, Ep.Steps: 483, Score: 797.319, Avg.Score: 1247.775, Max.Score: 1898.410, Time: 09:54:16
Ep.: 2618, Total Steps: 1192115, Ep.Steps: 1000, Score: 1655.732, Avg.Score: 1245.896, Max.Score: 1898.410, Time: 09:54:46
Ep.: 2619, Total Steps: 1193115, Ep.Steps: 1000, Score: 1684.074, Avg.Score: 1244.332, Max.Score: 1898.410, Time: 09:55:17
Ep.: 2620, Total Steps: 1194115, Ep.Steps: 1000, Score: 1669.741, Avg.Score: 1242.478, Max.Score: 1898.410, Time: 09:55:47
Ep.: 2621, Total Steps: 1195115, Ep.Steps: 1000, Score: 1708.091, Avg.Score: 1241.113, Max.Score: 1898.410, Time: 09:56:16
Ep.: 2622, Total Steps: 

Ep.: 2681, Total Steps: 1248988, Ep.Steps: 872, Score: 1606.358, Avg.Score: 1276.263, Max.Score: 1913.717, Time: 10:21:43
Ep.: 2682, Total Steps: 1249295, Ep.Steps: 307, Score: 551.214, Avg.Score: 1262.791, Max.Score: 1913.717, Time: 10:21:53
Ep.: 2683, Total Steps: 1250252, Ep.Steps: 957, Score: 1758.274, Avg.Score: 1271.618, Max.Score: 1913.717, Time: 10:22:21
Ep.: 2684, Total Steps: 1251252, Ep.Steps: 1000, Score: 1888.763, Avg.Score: 1271.575, Max.Score: 1913.717, Time: 10:22:52
Ep.: 2685, Total Steps: 1251907, Ep.Steps: 655, Score: 1192.264, Avg.Score: 1283.351, Max.Score: 1913.717, Time: 10:23:13
Ep.: 2686, Total Steps: 1252907, Ep.Steps: 1000, Score: 1874.647, Avg.Score: 1291.629, Max.Score: 1913.717, Time: 10:23:42
Ep.: 2687, Total Steps: 1253907, Ep.Steps: 1000, Score: 1908.203, Avg.Score: 1300.160, Max.Score: 1913.717, Time: 10:24:11
Ep.: 2688, Total Steps: 1254907, Ep.Steps: 1000, Score: 1850.587, Avg.Score: 1312.737, Max.Score: 1913.717, Time: 10:24:42
Ep.: 2689, Total Step

Ep.: 2748, Total Steps: 1309664, Ep.Steps: 456, Score: 851.263, Avg.Score: 1712.372, Max.Score: 1939.235, Time: 10:51:51
Ep.: 2749, Total Steps: 1310664, Ep.Steps: 1000, Score: 1829.179, Avg.Score: 1711.625, Max.Score: 1939.235, Time: 10:52:23
Ep.: 2750, Total Steps: 1311664, Ep.Steps: 1000, Score: 1812.996, Avg.Score: 1710.753, Max.Score: 1939.235, Time: 10:52:54
Ep.: 2751, Total Steps: 1312664, Ep.Steps: 1000, Score: 1927.333, Avg.Score: 1728.091, Max.Score: 1939.235, Time: 10:53:23
Ep.: 2752, Total Steps: 1313664, Ep.Steps: 1000, Score: 1844.608, Avg.Score: 1728.207, Max.Score: 1939.235, Time: 10:53:52
Ep.: 2753, Total Steps: 1314664, Ep.Steps: 1000, Score: 1862.999, Avg.Score: 1728.564, Max.Score: 1939.235, Time: 10:54:22
Ep.: 2754, Total Steps: 1315664, Ep.Steps: 1000, Score: 1873.351, Avg.Score: 1728.674, Max.Score: 1939.235, Time: 10:54:46
Ep.: 2755, Total Steps: 1316664, Ep.Steps: 1000, Score: 1880.982, Avg.Score: 1728.740, Max.Score: 1939.235, Time: 10:55:11
Ep.: 2756, Total S

Ep.: 2815, Total Steps: 1371895, Ep.Steps: 1000, Score: 1909.924, Avg.Score: 1715.818, Max.Score: 1947.012, Time: 11:22:42
Ep.: 2816, Total Steps: 1372828, Ep.Steps: 933, Score: 1791.905, Avg.Score: 1714.696, Max.Score: 1947.012, Time: 11:23:10
Ep.: 2817, Total Steps: 1373828, Ep.Steps: 1000, Score: 1906.751, Avg.Score: 1715.409, Max.Score: 1947.012, Time: 11:23:42
Ep.: 2818, Total Steps: 1374828, Ep.Steps: 1000, Score: 1938.798, Avg.Score: 1716.507, Max.Score: 1947.012, Time: 11:24:07
Ep.: 2819, Total Steps: 1375828, Ep.Steps: 1000, Score: 1921.469, Avg.Score: 1716.674, Max.Score: 1947.012, Time: 11:24:39
Ep.: 2820, Total Steps: 1376828, Ep.Steps: 1000, Score: 1877.434, Avg.Score: 1733.769, Max.Score: 1947.012, Time: 11:25:09
Ep.: 2821, Total Steps: 1377828, Ep.Steps: 1000, Score: 1865.158, Avg.Score: 1733.646, Max.Score: 1947.012, Time: 11:25:39
Ep.: 2822, Total Steps: 1378828, Ep.Steps: 1000, Score: 1805.243, Avg.Score: 1749.636, Max.Score: 1947.012, Time: 11:26:10
Ep.: 2823, Total 

In [ ]:
reward_round = round(np.max(scores), 2)
save(agent, 'dir_chk_lr0.0003', 'weights', 'final', str(reward_round))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

print('length of scores: ', len(scores), ', len of avg_scores: ', len(avg_scores))

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores, label="Score")
plt.plot(np.arange(1, len(avg_scores)+1), avg_scores, label="Avg on 100 episodes")
plt.legend(bbox_to_anchor=(1.05, 1)) 
plt.ylabel('Score')
plt.xlabel('Episodes #')
plt.show()

In [ ]:
def play(env, agent, num_episodes):
    
    state = env.reset()
    scores_deque = deque(maxlen=100)
    scores = []
    
    for i_episode in range(num_episodes + 1):
        
        state = env.reset()
        score = 0                    
        time_start = time.time()
        
        while True:
            
            action = agent.select_action(state, eval=True)
            env.render()
            next_state, reward, done, _ = env.step(action)
            score += reward 
            state = next_state
    
            if done:
                break
                
        s = (int)(time.time() - time_start)
        
        scores_deque.append(score)
        scores.append(score)    
        
        print('Episode {}\tAverage Score: {:.2f},\tScore: {:.2f} \tTime: {:02}:{:02}:{:02}'\
                  .format(i_episode, np.mean(scores_deque), score, s//3600, s%3600//60, s%60)) 
    

In [ ]:
play(env=env, agent=agent, num_episodes=5)

In [ ]:
env.close()